In [2]:
import torch
import os
import tiktoken
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import json
import os
from tqdm import tqdm
import tiktoken
import h5py
import numpy as np

device = 'mps'
enc = tiktoken.get_encoding('r50k_base')

In [49]:
import zstandard as zstd
import json

in_dir = "/Volumes/External/EleutherAI_ThePile_v1/pile/"
out_file = "/Users/jake/personal/monosemantic/data/pile_val.h5"

with h5py.File(out_file, 'w') as out_f:
    dataset = out_f.create_dataset('tokens', (0,), maxshape=(None,), dtype='i')
    start_index = 0
    # for filename in sorted(os.listdir(in_dir)):
    # for filename in "00.jsonl.zst 01.jsonl.zst 02.jsonl.zst".split():
    for filename in ["val.jsonl.zst"]:
        if filename in ['.DS_Store']:
            continue
        in_file = os.path.join(in_dir, filename)
        print(in_file)
        with zstd.open(in_file, 'r') as in_f:
            for line in tqdm(in_f, total=7021450):
                data = json.loads(line)
                text = data['text'] + "<|endoftext|>"
                encoded = enc.encode(text, allowed_special={'<|endoftext|>'})
                encoded_len = len(encoded)
                end_index = start_index + encoded_len
                dataset.resize(dataset.shape[0] + encoded_len, axis=0)
                dataset[start_index:end_index] = encoded
                start_index = end_index



/Volumes/External/EleutherAI_ThePile_v1/pile/val.jsonl.zst


  3%|▉                             | 214670/7021450 [02:04<1:05:32, 1730.87it/s]


In [ ]:
for fname, text_split in zip(['data/pile_train.h5', 'data/pile_dev.h5'], [train_text, dev_text]):
    with h5py.File(fname, 'w') as f:
        dataset = f.create_dataset('indices', (0,), maxshape=(None,), dtype='i')
        # Fill the dataset in chunks
        start_index = 0
        for i in tqdm(range((len(text_split) // chunk_size) + 1)):
            encoded_chunk = enc.encode(text[i*chunk_size:(i+1)*chunk_size])
            chunk_len = len(encoded_chunk)
            end_index = start_index + chunk_len
            dataset.resize(dataset.shape[0] + chunk_len, axis=0)
            dataset[start_index:end_index] = encoded_chunk
            start_index = end_index

In [42]:
in_dir = "/Volumes/External/EleutherAI_ThePile_v1/pile/train/"


for filename in sorted(os.listdir(in_dir)):
    if filename in ['.DS_Store']:
        continue
    in_file = os.path.join(in_dir, filename)
    print(in_file)
    with zstd.open(in_file, 'r') as in_f:
        set_counts = {}
        for i, line in tqdm(enumerate(in_f), total=100000):
            data = json.loads(line)
            set_name = data['meta']['pile_set_name']
            if set_counts.get(set_name):
                set_counts[set_name] += 1
            else:
                set_counts[set_name] = 1
            if i > 100000:
                break
        total = sum(set_counts.values())
        print('\n'.join(sorted([f"{name}: {count/total*100:.4f}%" for name, count in set_counts.items()], key=lambda n: -float(n.split()[-1][:-1]))))

/Volumes/External/EleutherAI_ThePile_v1/pile/train/00.jsonl.zst


100001it [00:01, 71382.10it/s]                                                  


Pile-CC: 25.0095%
OpenWebText2: 15.4657%
StackExchange: 14.0007%
PubMed Abstracts: 13.8337%
Github: 8.6098%
Wikipedia (en): 7.9398%
USPTO Backgrounds: 5.3149%
PubMed Central: 2.6399%
FreeLaw: 2.4210%
ArXiv: 1.0920%
DM Mathematics: 0.8680%
NIH ExPorter: 0.8620%
HackerNews: 0.7290%
Enron Emails: 0.4590%
OpenSubtitles: 0.3140%
YoutubeSubtitles: 0.1560%
Books3: 0.1370%
EuroParl: 0.0530%
PhilPapers: 0.0340%
Gutenberg (PG-19): 0.0330%
BookCorpus2: 0.0170%
Ubuntu IRC: 0.0110%
/Volumes/External/EleutherAI_ThePile_v1/pile/train/01.jsonl.zst


100001it [00:01, 71815.82it/s]                                                  


Pile-CC: 24.8615%
OpenWebText2: 15.3167%
StackExchange: 14.1677%
PubMed Abstracts: 13.9197%
Github: 8.5998%
Wikipedia (en): 8.0498%
USPTO Backgrounds: 5.2689%
PubMed Central: 2.6749%
FreeLaw: 2.3280%
ArXiv: 1.1610%
DM Mathematics: 0.8990%
NIH ExPorter: 0.8440%
HackerNews: 0.7260%
Enron Emails: 0.4580%
OpenSubtitles: 0.2840%
YoutubeSubtitles: 0.1650%
Books3: 0.1400%
EuroParl: 0.0470%
PhilPapers: 0.0370%
Gutenberg (PG-19): 0.0280%
BookCorpus2: 0.0150%
Ubuntu IRC: 0.0090%
/Volumes/External/EleutherAI_ThePile_v1/pile/train/02.jsonl.zst


...

Pile-CC: 24.9685%
OpenWebText2: 15.4247%
StackExchange: 14.0857%
PubMed Abstracts: 13.8277%
Github: 8.4878%
Wikipedia (en): 8.1338%
USPTO Backgrounds: 5.2139%
PubMed Central: 2.6769%
FreeLaw: 2.3920%
ArXiv: 1.1590%
DM Mathematics: 0.8940%
NIH ExPorter: 0.7990%
HackerNews: 0.7800%
Enron Emails: 0.4360%
OpenSubtitles: 0.3160%
YoutubeSubtitles: 0.1490%
Books3: 0.1160%
EuroParl: 0.0580%
PhilPapers: 0.0340%
Gutenberg (PG-19): 0.0260%
Ubuntu IRC: 0.0120%
BookCorpus2: 0.0100%
/Volumes/External/EleutherAI_ThePile_v1/pile/train/06.jsonl.zst


100001it [00:05, 17297.24it/s]                                                  


Pile-CC: 24.9535%
OpenWebText2: 15.4467%
PubMed Abstracts: 14.0207%
StackExchange: 13.9427%
Github: 8.4598%
Wikipedia (en): 8.0488%
USPTO Backgrounds: 5.2899%
PubMed Central: 2.7219%
FreeLaw: 2.2810%
ArXiv: 1.1280%
DM Mathematics: 0.9080%
NIH ExPorter: 0.8620%
HackerNews: 0.7370%
Enron Emails: 0.4520%
OpenSubtitles: 0.3190%
YoutubeSubtitles: 0.1490%
Books3: 0.1390%
EuroParl: 0.0640%
Gutenberg (PG-19): 0.0330%
PhilPapers: 0.0250%
Ubuntu IRC: 0.0110%
BookCorpus2: 0.0080%
/Volumes/External/EleutherAI_ThePile_v1/pile/train/07.jsonl.zst


 35%|███████████▌                     | 35062/100000 [00:02<00:04, 15702.79it/s]


KeyboardInterrupt: 

In [26]:
set_counts

{'Pile-CC': 341014,
 'Github': 116996,
 'OpenWebText2': 210866,
 'StackExchange': 191107,
 'Wikipedia (en)': 109756,
 'PubMed Abstracts': 189450,
 'USPTO Backgrounds': 72053,
 'FreeLaw': 32604,
 'PubMed Central': 36868,
 'Enron Emails': 5930,
 'HackerNews': 10057,
 'NIH ExPorter': 11515,
 'Books3': 1841,
 'ArXiv': 15493,
 'DM Mathematics': 12310,
 'OpenSubtitles': 4054,
 'BookCorpus2': 170,
 'Ubuntu IRC': 119,
 'YoutubeSubtitles': 2200,
 'EuroParl': 826,
 'PhilPapers': 430,
 'Gutenberg (PG-19)': 431}

In [30]:
total = sum(set_counts.values())

{name: f"{count/total*100:.4f}%" for name, count in set_counts.items()}

{'Pile-CC': '24.9628%',
 'Github': '8.5643%',
 'OpenWebText2': '15.4357%',
 'StackExchange': '13.9893%',
 'Wikipedia (en)': '8.0343%',
 'PubMed Abstracts': '13.8680%',
 'USPTO Backgrounds': '5.2744%',
 'FreeLaw': '2.3867%',
 'PubMed Central': '2.6988%',
 'Enron Emails': '0.4341%',
 'HackerNews': '0.7362%',
 'NIH ExPorter': '0.8429%',
 'Books3': '0.1348%',
 'ArXiv': '1.1341%',
 'DM Mathematics': '0.9011%',
 'OpenSubtitles': '0.2968%',
 'BookCorpus2': '0.0124%',
 'Ubuntu IRC': '0.0087%',
 'YoutubeSubtitles': '0.1610%',
 'EuroParl': '0.0605%',
 'PhilPapers': '0.0315%',
 'Gutenberg (PG-19)': '0.0315%'}